# ResNet Retraining
## Seminario de Tesis I, Primavera 2022 
### MDS Program. University of Chile.
#### Supervisor: Prof. Benjamín Bustos, Prof. Iván Sipirán
#### Author: Iván Sipirán, modified by Matías Vergara


## Imports

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, models, transforms
import time
import os
import copy
import pandas as pd
import math
import random
import shutil

from torch.utils.data import Dataset
from PIL import Image

from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import numpy as np, scipy.io
import argparse
import json

## Mounting Google Drive

In [2]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    folder_path = 'drive/MyDrive/TesisMV/'
except:
    folder_path = '../'

## Dataset and model selection

In [24]:
#modify only this cell
USE_RN50 = True
DS_FLAGS = []
              # 'ref': [invertX, invertY],
              # 'rot': [rotate90, rotate180, rotate270],
              # 'crop': [crop] * CROP_TIMES,
              # 'blur': [blur],
              # 'emboss': [emboss],
              # 'randaug': [randaug],
              # 'rain': [rain],
              # 'elastic': [elastic]
CROP_TIMES = 2
RANDOM_TIMES = 2
ELASTIC_TIMES = 2

In [25]:
# This cells builds the data_flags variable, that will be used
# to map the requestes data treatment to folders
MAP_TIMES = {'crop': CROP_TIMES,
         'randaug': RANDOM_TIMES,
         'elastic': ELASTIC_TIMES,
}

DS_FLAGS = sorted(DS_FLAGS)
data_flags = '_'.join(DS_FLAGS) if len(DS_FLAGS) > 0 else 'base'
MULTIPLE_TRANSF = ['crop', 'randaug', 'elastic']
COPY_FLAGS = DS_FLAGS.copy()

for t in MULTIPLE_TRANSF:
    if t in DS_FLAGS:
        COPY_FLAGS.remove(t)
        COPY_FLAGS.append(t + str(MAP_TIMES[t]))
        data_flags = '_'.join(COPY_FLAGS)

patterns_path = folder_path + "patterns/" + data_flags
labels_path = folder_path + "labels/" + data_flags
if not (os.path.isdir(patterns_path) and os.path.isdir(labels_path)):
    raise FileNotFoundError("No existen directorios de datos para el conjunto de flags seleccionado. Verifique que el dataset exista y, de lo contrario, llame a Split and Augmentation")
print("Pattern set encontrado en {}".format(patterns_path))
print("Labels set encontrado en {}".format(labels_path))
OUTPUT_FILENAME = f'resnet50_{data_flags}.pth' if USE_RN50 else f'resnet18_{data_flags}.pth'

Pattern set encontrado en ../patterns/base
Labels set encontrado en ../labels/base


In [26]:
model_output_path = 'models/' + OUTPUT_FILENAME

## Transfer Learning

In [19]:
pathDataset = patterns_path + '/'

train_dataset = torchvision.datasets.ImageFolder(pathDataset + 'train', 
                                                    transform = transforms.Compose([
                                                        transforms.RandomVerticalFlip(),
                                                        transforms.RandomHorizontalFlip(),
                                                        transforms.RandomResizedCrop(224),
                                                                    transforms.ToTensor(),
                                                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                                        std = [0.229, 0.224, 0.225])]))

val_dataset = torchvision.datasets.ImageFolder(pathDataset + 'val',
                                                    transform = transforms.Compose([ transforms.Resize(256),
                                                                    transforms.CenterCrop(224),
                                                                    transforms.ToTensor(),
                                                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                                        std = [0.229, 0.224, 0.225])]))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32,shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=True)

class_names = train_dataset.classes

device = ('cuda' if torch.cuda.is_available() else 'cpu')

def train_model(model, criterion, optimizer, num_epochs=30, output_path = 'model.pth', save_each = -1):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs-1}')
        print('-' * 10)

        model.train()

        running_loss = 0.0
        running_corrects = 0.0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds ==  labels.data)
        
        epoch_loss = running_loss / len(train_dataset)
        epoch_acc = running_corrects.double() / len(train_dataset)

        print('Train Loss: {:.4f}  Acc: {:.4f}'.format(epoch_loss, epoch_acc))

        #Validation
        model.eval()
        running_loss = 0.0
        running_corrects = 0.0

        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            with torch.set_grad_enabled(False):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
            
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(val_dataset)
        epoch_acc = running_corrects / len(val_dataset)
        print('Val Loss: {:.4f}  Acc: {:.4f}'.format(epoch_loss, epoch_acc))

        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())

        if save_each > -1 and epoch%save_each == 0:
            path = output_path.split("/")
            filename =  path[-1]
            epoch_filename =filename.split(".")[0] + "_e" + str(epoch) + "." + filename.split(".")[1]
            new_path = path[:-1]
            new_path.append(epoch_filename)
            new_path = '/'.join(new_path)
            torch.save(model.state_dict(), new_path)
            print("Saving model at epoch {} as {}".format(epoch, new_path))

    print('Best accuracy: {:.4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)

    return model

In [20]:
device

'cuda'

In [21]:
if USE_RN50:
    model_ft = models.resnet50(pretrained=True)
else:
    model_ft = models.resnet18(pretrained=True)
num_ft = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ft, 6)

model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()

learning_rate = 0.001
groups = [{'params': model_ft.conv1.parameters(),'lr':learning_rate/4},
            {'params': model_ft.bn1.parameters(),'lr':learning_rate/4},
            {'params': model_ft.layer1.parameters(),'lr':learning_rate/4},
            {'params': model_ft.layer2.parameters(),'lr':learning_rate/2},
            {'params': model_ft.layer3.parameters(), 'lr':learning_rate/2},
            {'params': model_ft.layer4.parameters(),'lr':learning_rate},
            {'params': model_ft.fc.parameters(), 'lr':learning_rate}]

optimizer = torch.optim.Adam(model_ft.parameters(), lr = 0.002)

output_path = folder_path + model_output_path

# change save each and output_path
model_ft = train_model(model_ft, criterion, optimizer, num_epochs=100, save_each=5, output_path = output_path)

# train_model returns the best model, so save it
# torch.save(model_ft.state_dict(), output_path)

Epoch 0/99
----------
Train Loss: 1.9553  Acc: 0.3952
Val Loss: 16.0777  Acc: 0.2949
Saving model at epoch 0 as ../models/resnet18_base_e0.pth
Epoch 1/99
----------
Train Loss: 1.4057  Acc: 0.4581
Val Loss: 1.4573  Acc: 0.5128
Epoch 2/99
----------
Train Loss: 1.4092  Acc: 0.4613
Val Loss: 1.3735  Acc: 0.5128
Epoch 3/99
----------
Train Loss: 1.1908  Acc: 0.5548
Val Loss: 1.2308  Acc: 0.5641
Epoch 4/99
----------
Train Loss: 1.1513  Acc: 0.5790
Val Loss: 1.3518  Acc: 0.5256
Epoch 5/99
----------
Train Loss: 1.1678  Acc: 0.5839
Val Loss: 1.1450  Acc: 0.6154
Saving model at epoch 5 as ../models/resnet18_base_e5.pth
Epoch 6/99
----------
Train Loss: 1.1179  Acc: 0.5952
Val Loss: 1.2627  Acc: 0.6154
Epoch 7/99
----------
Train Loss: 1.0554  Acc: 0.6145
Val Loss: 0.9183  Acc: 0.7179
Epoch 8/99
----------
Train Loss: 1.0755  Acc: 0.6306
Val Loss: 1.2660  Acc: 0.6026
Epoch 9/99
----------
Train Loss: 1.0217  Acc: 0.6532
Val Loss: 1.0404  Acc: 0.6538
Epoch 10/99
----------
Train Loss: 0.9266  

## Testing Transfer Learning

In [43]:
model = '../' + model_output_path
#model = '../' + 'models/resnet18_base_each5/resnet18_base_e90.pth'

pathDataset = patterns_path + '/'

test_dataset = torchvision.datasets.ImageFolder(pathDataset + 'test',
                                                    transform = transforms.Compose([ transforms.Resize(224),
                                                                    #transforms.CenterCrop(224),
                                                                    transforms.ToTensor(),
                                                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                                        std = [0.229, 0.224, 0.225])]))

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=True)
device = ('cuda' if torch.cuda.is_available() else 'cpu')

if USE_RN50:
    model_ft = models.resnet50(pretrained=True)
else:
    model_ft = models.resnet18(pretrained=True)

num_ft = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ft, 6)

model_ft = model_ft.to(device)

model_ft.load_state_dict(torch.load(model))
criterion = nn.CrossEntropyLoss()

model_ft.eval()
running_loss = 0.0
running_corrects = 0.0

for inputs, labels in test_loader:
    inputs = inputs.to(device)
    labels = labels.to(device)

    with torch.set_grad_enabled(False):
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

epoch_loss = running_loss / len(test_dataset)
epoch_acc = running_corrects / len(test_dataset)
print('Test Loss: {:.4f}  Acc: {:.4f}'.format(epoch_loss, epoch_acc))

Test Loss: 0.3862  Acc: 0.9103
